In [8]:
import datetime
import matplotlib.pyplot as plt
import os
import pandas as pd
from typing import Sequence

In [9]:
def read_partition(path: str) -> pd.DataFrame:
    def _read_partition(path: str, partition_cols: Sequence[Sequence[str]]) -> pd.DataFrame:
        if path.lower().endswith(".xlsx"):
            df = pd.read_excel(path)
            return df.assign(**dict(partition_cols))
        elif os.path.isdir(path):
            return pd.concat(
                [_read_partition(f"{path}/{subpath}", partition_cols + ([subpath.split("=")] if "=" in subpath else [])) for subpath in os.listdir(path)]
            )
    
    return _read_partition(path, [])


def parse_tamtop_datetime(input: str) -> datetime.datetime:
    return datetime.datetime.fromisoformat(
        input.replace(" ", "T").replace("(", "").replace(")", "")
    )

In [10]:
df = read_partition("data")
df["Time"] = df["Time"].apply(parse_tamtop_datetime)
df = df.set_index(["Device", "Time"]).drop(["NO."], axis=1)